## Import libraries

In [34]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

## Import and clean data

In [35]:
df_CCD = pd.read_csv("https://raw.githubusercontent.com/ferbracalente/E.-coli-coculture/master/docs/CCD.csv", index_col=("Std"))

cols = list(df_CCD.columns[2:7])
names = ("RQ5", "Time", "IPTG", "ug_MBE", "OD")

df_CCD = df_CCD[cols]
df_CCD.columns = names

df_CCD.head()

,RQ5,Time,IPTG,ug_MBE,OD
Std,,,,,
1,25.672741,1.310793,0.120944,12.153163,9.210
2,25.672741,1.310793,0.120944,12.231740,8.785
3,74.317259,1.310793,0.120944,5.648743,8.770
4,74.317259,1.310793,0.120944,5.072542,8.975
5,25.672741,3.689207,0.120944,2.726779,8.975


## Define the polynomial model for each response

In [36]:
def model_MBE(RQ5, Time, IPTG):
    MBE_predicted = -16.59126 -0.13119 * RQ5 -0.70722 * Time +285.59296 * IPTG + 0.39402 * RQ5*Time + 2.70772  * RQ5*IPTG + 60.23277 * Time * IPTG - 3.79661E-003 * (RQ5**2) -3.12603 * (Time**2) -1374.69579 * (IPTG**2) -1.09541 * RQ5*Time*IPTG -2.28001E-003 * (RQ5**2) *Time +1286.95684*(IPTG**3)
    return MBE_predicted

def model_OD(RQ5, Time, IPTG):
    OD_predicted = 13.49564 -0.090711 * RQ5 -6.35033 * Time -12.09413 * IPTG + 0.15325 * RQ5*Time +0.23355 * RQ5*IPTG +6.45031 * Time*IPTG - 7.88557E-004 * (RQ5**2) + 1.07671 * (Time**2) -21.94053 * (IPTG**2) - 0.10508 * RQ5*Time*IPTG -0.026465 * RQ5*(Time**2)
    return OD_predicted

def model_MBE_coded(RQ5, Time, IPTG):
    return 23.91 -2.50 *RQ5 -1.32 * Time +1.19 *IPTG -2.33 *RQ5*Time -0.078 *RQ5*IPTG +0.68 *Time*IPTG -5.62 * (RQ5**2) -4.42 * (Time**2) -5.48 * (IPTG**2) -3.30 * RQ5*Time*IPTG -1.60 * (RQ5**2)*Time +1.45 * (IPTG**3)

def model_OD_coded(RQ5, Time, IPTG):
    return 8.20 + 1.01 *RQ5 + 0.41 * Time - 0.76 * IPTG - 0.079 *RQ5*Time -0.074 *RQ5*IPTG +0.15 *Time*IPTG -0.47 *(RQ5**2) -0.35 *(Time**2) -0.24 *(IPTG**2) -0.32 *RQ5*Time*IPTG - 0.91 * RQ5* (Time**2)

## Define range of idependent variables

In [37]:
RQ5 = np.linspace(0, 100, 1000)
Time = np.linspace(0.5, 4.5, 1000)
IPTG = np.linspace(0.05, 0.4, 1000)

## Define dictionaries with units, zero values and D_optimum for each factor

In [38]:
variables_dict = {'RQ5.1': RQ5, 'Time': Time, 'IPTG': IPTG}
units_dict = {'RQ5.1':"(%)", "Time": "(h)", "IPTG": "(mM)"}
zero_values_dict = {'RQ5.1':50, 'Time': 2.5, 'IPTG': 0.23}
D_optimum_values = {'RQ5.1': 53.9, 'Time':2.51, 'IPTG':0.21}
responses_dict = {"MBE": model_MBE, "OD": model_OD}

## Define function to create a 2d-array with predicted responses

In [39]:
def predicted_responses(a:str, b:str, c:tuple, response_function):
    '''
    Recieves the name of two factors as strings, a tuple with the third factor and its fixed value 
    and the response fuction model.
    Return R, a 2d-array as result of aplying response_fuction(a, b, c=fix)
    ''' 
    A, B = np.meshgrid(variables_dict[a],variables_dict[b]) #Creates 2d-arrays A and B
    if (a == "RQ5.1" and b == "Time"):
        R = response_function(RQ5 = A, Time = B, IPTG = c[1]) #Apply function to each (A,B) pair
    elif (a == "Time" and b == "RQ5.1"):
        R = response_function(Time = A, RQ5 = B, IPTG = c[1])
    elif (a == "RQ5.1" and b == "IPTG"):
        R = response_function(RQ5= A, IPTG = B, Time = c[1])
    elif (a == "IPTG" and b == "RQ5.1"):
        R = response_function(IPTG = A, RQ5 = B, Time = c[1])
    elif (a == "Time" and b == "IPTG"):
        R = response_function(Time = A, IPTG = B, RQ5 = c[1])
    else:
        R = response_function(IPTG = A, Time = B, RQ5 = c[1])
    R[R<0] = 0 #All predicted negative values are converted to 0
    return R

## Define individual desirability function for each response

In [40]:
def ind_desirability(response: str, response_matrix):
    '''
    Recieves a string ("MBE" or "OD") indicating the response to calcule its individual desirability function
    Recieves a 2D-array with the predicted responses for a pair of factors and apply a one-side transformation 
    to return a 2-array with desirabilities
    '''
    d = np.zeros(response_matrix.shape).flatten()
    response_flatten = response_matrix.copy().flatten()
    if response == "MBE":
        min_value = df_CCD["ug_MBE"].min()
        max_value = df_CCD["ug_MBE"].max()
    if response == "OD":
        min_value = df_CCD["OD"].min()
        max_value = df_CCD["OD"].max()
    
    # predicted values lower than the minimum observed value, are asigned d=0
    d[response_flatten <= min_value] = 0 
    # predicted values higher than maximum observed value, are asigned d=1
    d[response_flatten >= max_value] = 1
    # predicted values between the minimum and maximum observed values are asigned 0<d<1
    d[(response_flatten >= min_value) & (response_flatten <= max_value)] = (response_flatten[(response_flatten>= min_value) & (response_flatten <= max_value)] -min_value)/(max_value - min_value)
    d.resize(response_matrix.shape)
    return d

def global_desirability(d1, d2):
    '''
    Recieves a 2d-array with individual desirabilities for each response.
    Returns one 2d-array with global desirability D
    '''
    D = np.sqrt(d1*d2)
    return D

## Define functions to construct 3D-response surfaces with plotly

In [41]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'browser' #Shows the plot on a browser tab

def graph_titles(a:str, b:str, c:tuple, response:str):
    '''
    Recieves the name of two factors (a, b) as strings, a tuple with the third factor (c) and its fixed value, 
    and a response.
    Return title and axis labels of the response surface plot
    '''
    title = f'{a}{units_dict[a]} vs {b}{units_dict[b]}, {c[0]} = {c[1]} {units_dict[c[0]]}'
    xaxis = f'{a}{units_dict[a]}'
    yaxis = f'{b}{units_dict[b]}'
    if response == "MBE":
        zaxis = 'ug MBE/OD600/10'
    elif response == "OD":
        zaxis = 'OD600'
    elif response == "D":
        zaxis = "Desirability"
    return title, xaxis, yaxis, zaxis

def interactive_graph(a:str, b:str, c: tuple, response: str):
    ''' 
    Recieves the name of two factors (a, b) as strings, a tuple with the third factor (c) and its fixed value 
    and a response as str ("MBE", "OD" or "D").
    Return an interactive response surface plot for the pair of factos (a,b), maintaining c constant.
    '''
    A, B = np.meshgrid(variables_dict[a], variables_dict[b])
    if response == "D":
        MBE_D_optimum = predicted_responses(a, b, c, model_MBE) 
        OD_D_optimum = predicted_responses(a, b, c, model_OD)
        d_MBE = ind_desirability("MBE", MBE_D_optimum)
        d_OD = ind_desirability("OD", OD_D_optimum)
        R = global_desirability(d_MBE, d_OD)
    else:
        R = predicted_responses(a, b, c, responses_dict[response])
    fig = go.Figure(data=[go.Surface(x=A, y=B, z=R)])
    fig.update_traces(opacity = 0.5,
                      colorbar=dict(len=0.5, yanchor="middle", ypad = 10),
                      contours_z=dict(show=True, size= 100, usecolormap=True,
                                      highlightcolor="limegreen", project_z=True))
    fig.update_layout(title = graph_titles(a, b, c, response)[0],
                      scene= dict(xaxis_title = graph_titles(a, b, c, response)[1], 
                                  yaxis_title = graph_titles(a, b, c, response)[2],
                                  zaxis_title = graph_titles(a, b, c, response)[3]),
                      font=dict(family="Arial", size=10,
                                color="black"),
                      width=700, height=700,
                      margin=dict(l=65, r=50, b=65, t=90))
    fig.show()

## Interactive MBE Response surfaces

In [42]:
interactive_graph("RQ5.1", "Time", ("IPTG", 0.23), "MBE")

In [43]:
interactive_graph("RQ5.1", "IPTG", ("Time", 2.5), "MBE")

In [47]:
interactive_graph("Time", "IPTG", ("RQ5.1", 50), "MBE")

## Interactive OD Response surfaces

In [48]:
interactive_graph("RQ5.1", "Time", ("IPTG", 0.23), "OD")

In [49]:
interactive_graph("RQ5.1", "IPTG", ("Time", 2.5), "OD")

In [50]:
interactive_graph("Time", "IPTG", ("RQ5.1", 50), "OD")

## Interactive Global Desirability response surfaces

In [51]:
interactive_graph("RQ5.1", "Time", ("IPTG", D_optimum_values["IPTG"]), "D")

In [52]:
interactive_graph("RQ5.1", "IPTG", ("Time", D_optimum_values["Time"]), "D")

In [53]:
interactive_graph("Time", "IPTG", ("RQ5.1", D_optimum_values["RQ5.1"]), "D")